In [ ]:
### imports

# external modules
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import importlib

# internal modules
sys.path.append('datagen')
import datagen
importlib.reload(datagen)
sys.path.append('reco')
import reco
import counting
import plotting
importlib.reload(reco)
importlib.reload(counting)
importlib.reload(plotting)

In [ ]:
### generate an image and count the objects in it

# generate image
image_size = (256,256)
objects = {'blob': {1:10, 2:5, 10:3},
           'line': {15:4},
           'curve': {(15,0.25):2, (5,0.3):2} }
im = datagen.generate_image( image_size=image_size, objects=objects )

# reconstruct objects
reco_objects = counting.reco_objects(im)

# compare counts to true values
ntrue = sum([sum(el.values()) for el in objects.values()])
nreco = len(reco_objects)
print('True number of objects: {}'.format(ntrue))
print('Reconstructed number of objects: {}'.format(nreco))

# make a plot
(fig,ax) = plotting.reco_plot_default( im, reco_objects )

In [ ]:
### generate many images and compare the reconstruction to the true values

# settings
nimages = 1000
doplot = True
nplotsmax = 5

# initializations
nplots = 0
true_counts = {}
reco_counts = {}
shapes = ['dot','blob','line','total']
for shape in shapes:
    true_counts[shape] = np.zeros(nimages)
    reco_counts[shape] = np.zeros(nimages)
rng = np.random.default_rng()

# do image loop
for i in range(nimages):

    # generate image
    image_size = (256,256)
    ndots = int(rng.uniform()*15)
    nblobs1 = int(rng.uniform()*10)
    nblobs2 = int(rng.uniform()*5)
    nblobs = nblobs1+nblobs2
    nlines = int(rng.uniform()*4)
    ntotal = ndots + nblobs + nlines
    objects = {'blob': {1:ndots, 2:nblobs1, 10:nblobs2},
               'line': {15:nlines}}
    im = datagen.generate_image( image_size=image_size, objects=objects )
    true_counts['dot'][i] = ndots
    true_counts['blob'][i] = nblobs
    true_counts['line'][i] = nlines
    true_counts['total'][i] = ntotal
    
    # reconstruct objects
    reco = counting.reco_objects(im)
    ndots_reco = len([el for el in reco if el['type']=='dot'])
    nblobs_reco = len([el for el in reco if el['type']=='blob'])
    nlines_reco = len([el for el in reco if el['type']=='line'])
    ntotal_reco = len(reco)
    reco_counts['dot'][i] = ndots_reco
    reco_counts['blob'][i] = nblobs_reco
    reco_counts['line'][i] = nlines_reco
    reco_counts['total'][i] = ntotal_reco
    
    # make a plot in case the reconstruction was not perfect
    if( doplot and nplots<nplotsmax
        and (
             ndots != ndots_reco
             or nblobs != nblobs_reco
             or nlines != nlines_reco 
             )
        ): 
        print('Found error in reconstruction:')
        print('Dots')
        print('  True: {}'.format(ndots))
        print('  Reconstructed: {}'.format(ndots_reco))
        print('Blobs:')
        print('  True: {}'.format(nblobs))
        print('  Reconstructed: {}'.format(nblobs_reco))
        print('Lines:')
        print('  True: {}'.format(nlines))
        print('  Reconstructed: {}'.format(nlines_reco))
        print('Total:')
        print('  True: {}'.format(ntotal))
        print('  Reconstructed: {}'.format(ntotal_reco))

        (fig,ax) = plotting.reco_plot_default( im, reco )
        plt.show()
        nplots += 1

In [ ]:
### continuation of previous cell
# (make plots)

# make plot of true and reconstructed counts
fig,ax = plt.subplots(figsize=(18,4), ncols=4)
nmaxobjects = 30
bins = np.linspace(-0.5, nmaxobjects+0.5, num=nmaxobjects+2)
for i,shape in enumerate(shapes):
    ax[i].hist(true_counts[shape], bins=bins, histtype='step', 
               color='b', linewidth=2, label='True {} count'.format(shape))
    ax[i].hist(reco_counts[shape], bins=bins, histtype='step', 
               color='r', linewidth=2, label='Reconstructed {} count'.format(shape))
    # plot aesthetics
    ax[i].legend()
    ax[i].set_ylim((0,ax[i].get_ylim()[1]*1.3))
    ax[i].set_xlabel('Number of objects')
    ax[i].set_ylabel('Number of images')

# make plot of absolute error
fig,ax = plt.subplots()
cdict = {'dot':'r', 'blob':'g', 'line':'b', 'total':'k'}
bins = np.linspace(-0.5, 3.5, num=5)
for i,shape in enumerate(shapes):
    abserror = np.abs(true_counts[shape]-reco_counts[shape])
    ax.hist(abserror, bins=bins, histtype='step', 
            color=cdict[shape], linewidth=2, label='Absolute {} error'.format(shape))
ax.legend()
ax.set_yscale('log')
ax.set_xlabel('Absolute error')
ax.set_ylabel('Number of images')

# make plot of relative error
fig,ax = plt.subplots()
cdict = {'dot':'r', 'blob':'g', 'line':'b', 'total':'k'}
bins = np.linspace(0., 2., num=20)
for i,shape in enumerate(shapes):
    abserror = np.abs(true_counts[shape]-reco_counts[shape])
    denominator = np.where( true_counts[shape]>0, true_counts[shape], 1)
    relerror = np.divide(abserror,denominator)
    ax.hist(relerror, bins=bins, histtype='step', 
            color=cdict[shape], linewidth=2, label='Relative {} error'.format(shape))
ax.legend()
ax.set_yscale('log')
ax.set_xlabel('Relative error')
ax.set_ylabel('Number of images')

In [ ]:
### test on a real image

sys.path.append('reading')
import EVIFile as evif

# read the file
evi = evif.EVIRead('reading/example1.EVI')
im = evi.get_data()[:,:,0]

# crop the image (optional)
im = im[:,:256]

# reconstruct objects
reco_objects = counting.reco_objects(im)

# compare counts to true values
nreco = len(reco_objects)
print('Reconstructed number of objects: {}'.format(nreco))

# make a plot
(fig,ax) = plotting.reco_plot_default( im, reco_objects )